In [1]:
import pandas as pd
import numpy as np
from glob import glob
from dateutil.relativedelta import relativedelta ,TH
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [2]:
data_path = glob(r'C:\Users\anish.maurya\Desktop\Data/*')
fut_data_path = data_path[0]
option_data_path = data_path[1]
fut_data = pd.read_csv(fut_data_path)
fut_data.drop(fut_data.columns[5:],axis =1 , inplace = True)
fut_data['date'] = pd.to_datetime(fut_data['date'])
fut_data['Day'] = pd.to_datetime(fut_data['date']).dt.day
fut_data['Month']=pd.to_datetime(fut_data['date']).dt.month
fut_data['Year'] = pd.to_datetime(fut_data['date']).dt.year
fut_data['Time'] = pd.to_datetime(fut_data['date']).dt.time
spot_price = fut_data[(fut_data['Year'] == 2022) & (fut_data['Month'] == 4) &(fut_data['Day'] == 1)].reset_index(drop = 'Frist')
spot_price['Date'] = pd.to_datetime(spot_price['date']).dt.date
spot_price.drop(['date' , 'high' , 'low' , 'Day' , 'Month' , 'Year'] , axis =1 , inplace = True)
spot_price = spot_price.drop(index = [0,1],axis = 0).reset_index(drop = 'Frist')
entry_time = '09:17:59'
exit_time = '15:19:59'
trade_log = pd.DataFrame(columns = ['Date', 'Ltp','Entry_time', 'Atm', 'Expiry', 'Ce_strike_price', 
                      'Ce_entry_price', 'Ce_exit_price', 'Ce_pnl', 'Pe_strike_price', 'Pe_entry_price',
                      'Pe_exit_price', 'Pe_pnl', 'Exit_time'])
trade_log_data = trade_log.to_csv('trade_log' , index = False)



for index,row in spot_price.iterrows():
    try:
        ltp = row['open']
        time = row['Time']
        base_price = 50
        atm = base_price*round(ltp/base_price)
        print(index)
        print(f'Current Nifty price:- {ltp}')
        time = str(time)
        print(f'Time:- {time}')
        if atm not in trade_log['Atm'].values and time <=exit_time :

            df = pd.read_csv(option_data_path)
            df['Date'] = pd.to_datetime(df['Date'])
            date = df['Date'][0]
            nearest_exp = df['Date'].apply(lambda x: x.date() + relativedelta(weekday= TH))
            exp = str(nearest_exp[0])
            df['Expiry'] = df['Expiry'].replace('20221229' , '29DEC22')
            df['Expiry'] = pd.to_datetime(df['Expiry'])
            intrday_data = df[(df['Expiry'] ==nearest_exp)|(df['Expiry'] == nearest_exp - pd.DateOffset(days=1))|(df['Expiry'] == nearest_exp - pd.DateOffset(days=2))].reset_index(drop = 'Frist')
            intrday_data = intrday_data[intrday_data['Time'] > time].reset_index(drop = 'Frist')
            ce_entry_price = intrday_data[(intrday_data['Strike'] == atm) & (intrday_data['OptionType'] == 'CE')].reset_index(drop = 'Frist')['Open'][0]
            pe_entry_price = intrday_data[(intrday_data['Strike'] == atm) & (intrday_data['OptionType'] == 'PE')].reset_index(drop = 'Frist')['Open'][0]
            ce_exit_price = intrday_data[(intrday_data['Strike'] == atm) & (intrday_data['Time'] == exit_time) & (intrday_data['OptionType'] == 'CE')].reset_index(drop = 'Frist')['Close'][0]
            pe_exit_price = intrday_data[(intrday_data['Strike'] == atm) & (intrday_data['Time'] == exit_time) & (intrday_data['OptionType'] == 'PE')].reset_index(drop = 'Frist')['Close'][0]
            data =({'Date':date , 'Entry_time':time ,'Ltp':ltp ,'Atm':atm, 'Expiry':exp ,'Ce_strike_price':atm , 
                                  'Ce_entry_price':ce_entry_price , 'Ce_exit_price':ce_exit_price , 'Ce_pnl':(ce_entry_price -ce_exit_price ) , 'Pe_strike_price':atm, 'Pe_entry_price':pe_entry_price ,
                                  'Pe_exit_price':pe_exit_price , 'Pe_pnl':(pe_entry_price -pe_exit_price ) , 'Exit_time':exit_time})
            data = pd.DataFrame([data])
            trade_log = pd.concat([trade_log , data], ignore_index=True)
            trade_log['Net_pnl'] = trade_log['Ce_pnl'] + trade_log['Pe_pnl']
            trade_log.to_csv('trade_log' , index = False)

        else:
            print(f'already position created this strike price:- {atm}')
            
    except Exception as e:
        print(e)





0
Current Nifty price:- 17484.5
Time:- 09:21:00
1
Current Nifty price:- 17495.45
Time:- 09:24:00
already position created this strike price:- 17500
2
Current Nifty price:- 17479.4
Time:- 09:27:00
already position created this strike price:- 17500
3
Current Nifty price:- 17467.85
Time:- 09:30:00
4
Current Nifty price:- 17486.55
Time:- 09:33:00
already position created this strike price:- 17500
5
Current Nifty price:- 17485.7
Time:- 09:36:00
already position created this strike price:- 17500
6
Current Nifty price:- 17487.0
Time:- 09:39:00
already position created this strike price:- 17500
7
Current Nifty price:- 17479.45
Time:- 09:42:00
already position created this strike price:- 17500
8
Current Nifty price:- 17494.7
Time:- 09:45:00
already position created this strike price:- 17500
9
Current Nifty price:- 17500.45
Time:- 09:48:00
already position created this strike price:- 17500
10
Current Nifty price:- 17496.15
Time:- 09:51:00
already position created this strike price:- 17500
11
Cur

86
Current Nifty price:- 17577.4
Time:- 13:39:00
already position created this strike price:- 17600
87
Current Nifty price:- 17575.55
Time:- 13:42:00
already position created this strike price:- 17600
88
Current Nifty price:- 17582.7
Time:- 13:45:00
already position created this strike price:- 17600
89
Current Nifty price:- 17586.9
Time:- 13:48:00
already position created this strike price:- 17600
90
Current Nifty price:- 17581.5
Time:- 13:51:00
already position created this strike price:- 17600
91
Current Nifty price:- 17575.85
Time:- 13:54:00
already position created this strike price:- 17600
92
Current Nifty price:- 17577.05
Time:- 13:57:00
already position created this strike price:- 17600
93
Current Nifty price:- 17579.85
Time:- 14:00:00
already position created this strike price:- 17600
94
Current Nifty price:- 17582.0
Time:- 14:03:00
already position created this strike price:- 17600
95
Current Nifty price:- 17582.45
Time:- 14:06:00
already position created this strike price:- 1

In [3]:
trade_log

,Date,Ltp,Entry_time,Atm,Expiry,Ce_strike_price,Ce_entry_price,Ce_exit_price,Ce_pnl,Pe_strike_price,Pe_entry_price,Pe_exit_price,Pe_pnl,Exit_time,Net_pnl
0,2022-04-01,17484.50,09:21:00,17500,2022-04-07,17500,161.45,265.35,-103.90,17500,175.20,69.4,105.80,15:19:59,1.90
1,2022-04-01,17467.85,09:30:00,17450,2022-04-07,17450,174.95,303.60,-128.65,17450,151.25,58.0,93.25,15:19:59,-35.40
2,2022-04-01,17527.40,10:00:00,17550,2022-04-07,17550,142.05,229.40,-87.35,17550,159.80,83.0,76.80,15:19:59,-10.55
3,2022-04-01,17583.55,13:36:00,17600,2022-04-07,17600,147.90,194.80,-46.90,17600,151.75,99.0,52.75,15:19:59,5.85
4,2022-04-01,17633.10,15:03:00,17650,2022-04-07,17650,132.20,163.65,-31.45,17650,143.25,116.7,26.55,15:19:59,-4.90
5,2022-04-01,17687.55,15:18:00,17700,2022-04-07,17700,135.30,134.35,0.95,17700,141.50,138.4,3.10,15:19:59,4.05


In [4]:
trade_log['Net_pnl'].sum()

-39.05000000000008